### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#See the list of columns within newly combine dataframe
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [3]:
#See the list of columns from the school dataframe
student_data.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')

In [4]:
#See the list of columns from the student dataframe
school_data.columns

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')

In [5]:
#See the top 5 observations to get an idea of the columns within newly combine dataframe
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [6]:
#Identify my datatypes on the columns to prepare for any datatype changes that may be needed for calculations
school_data_complete.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [7]:
#Check my counts to see if there is any null values
school_data_complete.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
dtype: int64

In [8]:
# Copy to another dataframe for shorter name
school_all = school_data_complete.copy()

# Count the unique number of schools and add it to array of total schools
dst_total_schools = school_all["school_name"].nunique()

# Count the number of students and add it to array of total students
dst_total_students = school_all["student_name"].count()

# Sum the budget of the schools to get total budget and add it to array of total budget
dst_total_budget = (school_all["budget"].unique()).sum()

# Average the number of students math scores and add it to array of average math score
dst_avg_math_score = school_all["math_score"].mean()

# Average the number of students reading scores and add it to array of average reading score
dst_avg_read_score = school_all["reading_score"].mean()

# For a math passing score of 70 or above get the number students and get the total count
dst_pass_math = school_all.loc[school_all["math_score"] >= 70]["math_score"].count()
# Calculate the total % of students passing math
dst_pct_pass_math = (dst_pass_math/dst_total_students) * 100

# For a reading passing score of 70 or above get the number students and get the total count
dst_pass_read = school_all.loc[school_all["reading_score"] >= 70]["reading_score"].count()
# Calculate the total % of students passing reading
dst_pct_pass_read = (dst_pass_read/dst_total_students) * 100

# For a reading and math passing score of 70 or above get the number students and get the total count
dst_pass_overall = school_all.loc[(school_all["math_score"] >= 70) & (school_all["reading_score"] >= 70)]["Student ID"].count()
# Calculate the total % of students passing overall
dst_pct_pass_overall = (dst_pass_overall/dst_total_students) * 100



In [9]:
# Create a District Summary Dataframe to house my district summary
district_summary = pd.DataFrame({
    
    "Total Schools": [dst_total_schools],
    "Total Students": [dst_total_students],
    "Total Budget": [dst_total_budget],
    "Average Math Score": [dst_avg_math_score],
    "Average Reading Score": [dst_avg_read_score],
    "% Passing Math":[dst_pct_pass_math],
    "% Passing Reading": [dst_pct_pass_read],
    "% Overall Passing": [dst_pct_pass_overall]
})

# Format the District Summary Dataframe
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [10]:
# Create groupby dataframe from the merge dataframe to use for the school summary based on grouping of school name 
gb_schools = school_all.groupby(["school_name"])

# Capture school types from original school input dataframe
# Set the index to school name
sch_types = school_data.set_index("school_name")["type"]

# Use groupby dataframe to get the total students per school
sch_total_students = gb_schools["student_name"].count()

# Capture the original budgets outlined in the school input dataframe
# Set the index to school name
sch_total_budget = school_data.set_index("school_name")["budget"]

# Capture the budgets per student using the school budgets over the students counts
sch_per_std_bud = sch_total_budget/sch_total_students

# Use groupby dataframe to get the average math score per school
sch_avg_math_score = gb_schools["math_score"].mean()

# Use groupby dataframe to get the average math score per school
sch_avg_read_score = gb_schools["reading_score"].mean()

# For a math passing score of 70 or above get the number students
# Then create groupby dataframe to get the count per school 
sch_pass_math = school_all.loc[school_all["math_score"] >= 70].groupby(["school_name"])["math_score"].count()

# Calculate the total % of students passing reading per school
sch_pct_pass_math = (sch_pass_math/sch_total_students) * 100

# For a reading passing score of 70 or above get the number students
# Then create groupby dataframe to get the count per school 
sch_pass_read = school_all.loc[school_all["reading_score"] >= 70].groupby(["school_name"])["reading_score"].count()

# Calculate the total % of students passing reading per school
sch_pct_pass_read = (sch_pass_read/sch_total_students) * 100

# For a reading and math passing score of 70 or above get the number students
# Then create groupby dataframe to get the count per school
sch_pass_overall = school_all.loc[(school_all["math_score"] >= 70) & (school_all["reading_score"] >= 70)].groupby(["school_name"])["Student ID"].count()

# Calculate the total % of students passing overall per school
sch_pct_pass_overall = (sch_pass_overall/sch_total_students) * 100


In [11]:
# Create a School Summary Dataframe to house my school summary
school_summary = pd.DataFrame({
    
    "School Type": sch_types,
    "Total Students": sch_total_students,
    "Total School Budget": sch_total_budget,
    "Per Student Budget": sch_per_std_bud,
    "Average Math Score": sch_avg_math_score,
    "Average Reading Score": sch_avg_read_score,
    "% Passing Math":sch_pct_pass_math,
    "% Passing Reading": sch_pct_pass_read,
    "% Overall Passing": sch_pct_pass_overall
})

# Format the School Summary Dataframe
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:.2f}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}".format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map("{:.2f}".format)
school_summary 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [12]:
# Sort top performing schools by % overall passing.
top_ovr_pass = school_summary.sort_values("% Overall Passing", ascending = False)

# Display the top five performing schools
top_ovr_pass.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [13]:
# Sort bottom performing schools by % overall passing.
bottom_ovr_pass = school_summary.sort_values("% Overall Passing", ascending = True)

# Display the bottom five performing schools
bottom_ovr_pass.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [14]:
# Capture all the 9th grade students
# Then create groupby dateframe to get the average (mean) math grade per school 
g9th_math = school_all.loc[school_all["grade"] == "9th"].groupby(["school_name"])["math_score"].mean()

# Capture all the 10th grade students
# Then create groupby dateframe to get the average (mean) math grade per school 
g10th_math = school_all.loc[school_all["grade"] == "10th"].groupby(["school_name"])["math_score"].mean()

# Capture all the 11th grade students
# Then create groupby dateframe to get the average (mean) math grade per school 
g11th_math = school_all.loc[school_all["grade"] == "11th"].groupby(["school_name"])["math_score"].mean()

# Capture all the 12th grade students
# Then create groupby dateframe to get the average (mean) math grade per school 
g12th_math = school_all.loc[school_all["grade"] == "12th"].groupby(["school_name"])["math_score"].mean()


In [15]:
# Create Average Math Score by students grade dataframe
math_scores_grade = pd.DataFrame({
        "9th": g9th_math,
        "10th": g10th_math,
        "11th": g11th_math,
        "12th": g12th_math
})

# Format the Average Math Score by Grade Summary Dataframe
math_scores_grade.index.name = None
math_scores_grade["9th"] = math_scores_grade["9th"].map("{:.2f}".format)
math_scores_grade["10th"] = math_scores_grade["10th"].map("{:.2f}".format)
math_scores_grade["11th"] = math_scores_grade["11th"].map("{:.2f}".format)
math_scores_grade["12th"] = math_scores_grade["12th"].map("{:.2f}".format)
math_scores_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [16]:
# Capture all the 9th grade students
# Then create groupby dateframe to get the average (mean) reading grade per school 
g9th_read = school_all.loc[school_all["grade"] == "9th"].groupby(["school_name"])["reading_score"].mean()

# Capture all the 10th grade students
# Then create groupby dateframe to get the average (mean) reading grade per school 
g10th_read = school_all.loc[school_all["grade"] == "10th"].groupby(["school_name"])["reading_score"].mean()

# Capture all the 11th grade students
# Then create groupby dateframe to get the average (mean) reading grade per school 
g11th_read = school_all.loc[school_all["grade"] == "11th"].groupby(["school_name"])["reading_score"].mean()

# Capture all the 12th grade students
# Then create groupby dateframe to get the average (mean) reading grade per school 
g12th_read = school_all.loc[school_all["grade"] == "12th"].groupby(["school_name"])["reading_score"].mean()


In [17]:
# Create Average Reading Score by students grade dataframe
read_scores_grade = pd.DataFrame({
        "9th": g9th_read,
        "10th": g10th_read,
        "11th": g11th_read,
        "12th": g12th_read
})

# Format the Average Reading Score by Grade Summary Dataframe
read_scores_grade.index.name = None
read_scores_grade["9th"] = read_scores_grade["9th"].map("{:.2f}".format)
read_scores_grade["10th"] = read_scores_grade["10th"].map("{:.2f}".format)
read_scores_grade["11th"] = read_scores_grade["11th"].map("{:.2f}".format)
read_scores_grade["12th"] = read_scores_grade["12th"].map("{:.2f}".format)
read_scores_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
# Check for types within school summary
school_summary.dtypes

School Type              object
Total Students            int64
Total School Budget      object
Per Student Budget       object
Average Math Score       object
Average Reading Score    object
% Passing Math           object
% Passing Reading        object
% Overall Passing        object
dtype: object

In [19]:
# Use copy of school summary
spend_range_bin=school_summary.copy()

# For Bin Ranges
spend_range_bin["Per Student Budget"].min()
spend_range_bin["Per Student Budget"].max()

# Convert my columns back to numbers to use for calculations
spend_range_bin["Per Student Budget"] = spend_range_bin["Per Student Budget"].str.replace("$", '', regex=True).astype('float')
spend_range_bin["Average Math Score"] = pd.to_numeric(spend_range_bin["Average Math Score"])
spend_range_bin["Average Reading Score"] = pd.to_numeric(spend_range_bin["Average Reading Score"])
spend_range_bin["% Passing Math"] = pd.to_numeric(spend_range_bin["% Passing Math"])
spend_range_bin["% Passing Reading"] = pd.to_numeric(spend_range_bin["% Passing Reading"])
spend_range_bin["% Overall Passing"] = pd.to_numeric(spend_range_bin["% Overall Passing"])

# Create my Bin Ranges
spr_bins = [0, 584.99, 629.99, 644.99, 680]

# Create my Bin Labels
spr_bin_labels = ["<$585", "$585-630", "$630-645", "$645-$680"]

# Create Spending Ranges (Per Student) column 
spend_range_bin["Spending Ranges (Per Student)"] = pd.cut(spend_range_bin["Per Student Budget"], bins = spr_bins, labels = spr_bin_labels)

# Create a Spending Ranges groupby dataframe
spend_range_gb = spend_range_bin.groupby(["Spending Ranges (Per Student)"])

# Capture values to put in summary of spending ranges per student dataframe
spr_avg_math_score = spend_range_gb["Average Math Score"].mean()
spr_avg_read_score = spend_range_gb["Average Reading Score"].mean()
spr_pass_math = spend_range_gb["% Passing Math"].mean()
spr_pass_read = spend_range_gb["% Passing Reading"].mean()
spr_pass_overall = spend_range_gb["% Overall Passing"].mean()


In [20]:
# Create Spending Range per students Summary dataframe
spend_range_std_sum = pd.DataFrame({
        "Average Math Score": spr_avg_math_score,
        "Average Reading Score": spr_avg_read_score,
        "% Passing Math": spr_pass_math,
        "% Passing Reading": spr_pass_read,
        "% Overall Passing": spr_pass_overall,
})

# Format Spending Range per students Summary Dataframe
spend_range_std_sum.index.name = "Spending Ranges (Per Student)"
spend_range_std_sum["Average Math Score"] = spend_range_std_sum["Average Math Score"].map("{:.2f}".format)
spend_range_std_sum["Average Reading Score"] = spend_range_std_sum["Average Reading Score"].map("{:.2f}".format)
spend_range_std_sum["% Passing Math"] = spend_range_std_sum["% Passing Math"].map("{:.2f}".format)
spend_range_std_sum["% Passing Reading"] = spend_range_std_sum["% Passing Reading"].map("{:.2f}".format)
spend_range_std_sum["% Overall Passing"] = spend_range_std_sum["% Overall Passing"].map("{:.2f}".format)
spend_range_std_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-$680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [21]:
# Use copy of school summary
sch_size_bin=school_summary.copy()

# Add school size column to dataframe
sch_size_bin["Total Size"] = school_data.set_index("school_name")["size"]

# For Bin Ranges
sch_size_bin["Total Size"].min()
sch_size_bin["Total Size"].max()

# Convert my columns back to numbers to use for calculations
sch_size_bin["Average Math Score"] = pd.to_numeric(sch_size_bin["Average Math Score"])
sch_size_bin["Average Reading Score"] = pd.to_numeric(sch_size_bin["Average Reading Score"])
sch_size_bin["% Passing Math"] = pd.to_numeric(sch_size_bin["% Passing Math"])
sch_size_bin["% Passing Reading"] = pd.to_numeric(sch_size_bin["% Passing Reading"])
sch_size_bin["% Overall Passing"] = pd.to_numeric(sch_size_bin["% Overall Passing"])

# Create my Bin Ranges
ss_bins = [0, 999.99, 1999.99, 5000]

# Create my Bin Labels
ss_bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create School Size column 
sch_size_bin["School Size"] = pd.cut(sch_size_bin["Total Size"], bins = ss_bins, labels = ss_bin_labels)

# Create a School Size groupby dataframe
sch_size_gb = sch_size_bin.groupby(["School Size"])

# Capture values to put in summary of school size dataframe
ss_avg_math_score = sch_size_gb["Average Math Score"].mean()
ss_avg_read_score = sch_size_gb["Average Reading Score"].mean()
ss_pass_math = sch_size_gb["% Passing Math"].mean()
ss_pass_read = sch_size_gb["% Passing Reading"].mean()
ss_pass_overall = sch_size_gb["% Overall Passing"].mean()

In [22]:
# Create School Size Summary dataframe
sch_size_sum = pd.DataFrame({
        "Average Math Score": ss_avg_math_score,
        "Average Reading Score": ss_avg_read_score,
        "% Passing Math": ss_pass_math,
        "% Passing Reading": ss_pass_read,
        "% Overall Passing": ss_pass_overall,
})

# Format School Size Summary Dataframe
sch_size_sum.index.name = "School Size"
sch_size_sum["Average Math Score"] = sch_size_sum["Average Math Score"].map("{:.2f}".format)
sch_size_sum["Average Reading Score"] = sch_size_sum["Average Reading Score"].map("{:.2f}".format)
sch_size_sum["% Passing Math"] = sch_size_sum["% Passing Math"].map("{:.2f}".format)
sch_size_sum["% Passing Reading"] = sch_size_sum["% Passing Reading"].map("{:.2f}".format)
sch_size_sum["% Overall Passing"] = sch_size_sum["% Overall Passing"].map("{:.2f}".format)
sch_size_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.89
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [23]:
# Use copy of school summary
dst_type_bin=school_summary.copy()


# Convert my columns back to numbers to use for calculations
dst_type_bin["Average Math Score"] = pd.to_numeric(dst_type_bin["Average Math Score"])
dst_type_bin["Average Reading Score"] = pd.to_numeric(dst_type_bin["Average Reading Score"])
dst_type_bin["% Passing Math"] = pd.to_numeric(dst_type_bin["% Passing Math"])
dst_type_bin["% Passing Reading"] = pd.to_numeric(dst_type_bin["% Passing Reading"])
dst_type_bin["% Overall Passing"] = pd.to_numeric(dst_type_bin["% Overall Passing"])


# Create a School Type groupby dataframe
dst_type_gb = dst_type_bin.groupby(["School Type"])

# Capture values to put in summary of School Type dataframe
dst_avg_math_score = dst_type_gb["Average Math Score"].mean()
dst_avg_read_score = dst_type_gb["Average Reading Score"].mean()
dst_pass_math = dst_type_gb["% Passing Math"].mean()
dst_pass_read = dst_type_gb["% Passing Reading"].mean()
dst_pass_overall = dst_type_gb["% Overall Passing"].mean()



In [24]:
# Create district Type School Summary dataframe
dst_type_sum = pd.DataFrame({
        "Average Math Score": dst_avg_math_score,
        "Average Reading Score": dst_avg_read_score,
        "% Passing Math": dst_pass_math,
        "% Passing Reading": dst_pass_read,
        "% Overall Passing": dst_pass_overall,
})

# Format district Type School Summary dataframe
dst_type_sum.index.name = "School Type"
dst_type_sum["Average Math Score"] = dst_type_sum["Average Math Score"].map("{:.2f}".format)
dst_type_sum["Average Reading Score"] = dst_type_sum["Average Reading Score"].map("{:.2f}".format)
dst_type_sum["% Passing Math"] = dst_type_sum["% Passing Math"].map("{:.2f}".format)
dst_type_sum["% Passing Reading"] = dst_type_sum["% Passing Reading"].map("{:.2f}".format)
dst_type_sum["% Overall Passing"] = dst_type_sum["% Overall Passing"].map("{:.2f}".format)
dst_type_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
